In [1]:
import os
import pickle
import requests
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, balanced_accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

2023-04-26 19:41:15.937143: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
boay = pd.read_csv('../../1-data-collection/data/boay00.csv')
cho = pd.read_csv('../../1-data-collection/data/cho_shmo.csv')


In [3]:
cho.drop(columns = 'Unnamed: 0', inplace = True)
boay.drop(columns = 'Unnamed: 0', inplace = True)

In [95]:
# res = requests.get('https://ballchasing.com/api/a402545e-5f1b-4446-b58d-9cb91b137aae',
#                   headers={
#                       'Authorization': 'mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5'})

# data = res.json()
# data

# player_data = data['orange']['players'][0]['stats']
# player_data = dict(player_data, **player_data['core'])
# player_data = dict(player_data, **player_data['boost'])
# player_data = dict(player_data, **player_data['movement'])
# player_data = dict(player_data, **player_data['positioning'])
# player_data = dict(player_data, **player_data['demo'])
# del player_data['core']
# del player_data['boost']
# del player_data['movement']
# del player_data['positioning']
# del player_data['demo']
# player_data['player_name'] = 'Vatira'



# df = pd.DataFrame(columns = player_data.keys())
# for j in range(0, min([len(data['orange']['players']), len(data['blue']['players'])])):
            
               
#     if data['orange']['players'][j]['name'].lower() == 'Vatira'.lower():
#         player_data = data['orange']['players'][j]['stats']

#         player_data = dict(player_data, **player_data['core'])
#         player_data = dict(player_data, **player_data['boost'])
#         player_data = dict(player_data, **player_data['movement'])
#         player_data = dict(player_data, **player_data['positioning'])
#         player_data = dict(player_data, **player_data['demo'])
#         del player_data['core']
#         del player_data['boost']
#         del player_data['movement']
#         del player_data['positioning']
#         del player_data['demo']

#         player_data['title'] = data['title']
#         player_data['player_name'] = 'Vatira'
#         df.loc[player_data['title'], :] = player_data

#     elif data['blue']['players'][j]['name'].lower() == 'Vatira'.lower():

#         player_data = data['blue']['players'][j]['stats']

#         player_data = dict(player_data, **player_data['core'])
#         player_data = dict(player_data, **player_data['boost'])
#         player_data = dict(player_data, **player_data['movement'])
#         player_data = dict(player_data, **player_data['positioning'])
#         player_data = dict(player_data, **player_data['demo'])
#         del player_data['core']
#         del player_data['boost']
#         del player_data['movement']
#         del player_data['positioning']
#         del player_data['demo']

#         player_data['title'] = data['title']
#         player_data['player_name'] = 'Vatira'

#         df.loc[player_data['title'], :] = player_data
# df.mvp = df.mvp.astype(int)
# df

{'chat': {'Goose': 'Great Pass!', 'YOU': 'gg.'},
 'error': 'not found',
 'request_id': '56B5475D:C7EC_A2379E0D:01BB_644962F6_A8BB87:31150',
 'server': 'hzprod-front-1'}

In [4]:
with open('../../3-modeling-playstyle/code/et_model.pkl', 'rb') as picklefile:
    et_model = pickle.load(picklefile)

In [5]:
with open('../../3-modeling-playstyle/code/nn_model.pkl', 'rb') as picklefile:
    nn_model = pickle.load(picklefile)

In [8]:
to_drop = [
    'shots_against',
    'goals_against',
    'shooting_percentage',
    'bpm',
    'amount_stolen_big',
    'amount_stolen_small',
    'count_collected_big',
    'count_collected_small',
    'count_stolen_small',
    'count_stolen_big',
    'amount_overfill_stolen',
    'time_zero_boost',
    'time_full_boost',
    'time_boost_0_25',
    'time_boost_25_50',
    'time_boost_50_75',
    'time_boost_75_100',
    'avg_speed',
    'total_distance',
    'time_supersonic_speed',
    'time_boost_speed',
    'time_slow_speed',
    'time_ground',
    'time_low_air',
    'time_high_air',
    'time_powerslide',
    'time_defensive_third',
    'time_neutral_third',
    'time_offensive_third',
    'time_defensive_half',
    'time_offensive_half',
    'time_behind_ball',
    'time_infront_ball',
    'time_most_back',
    'time_most_forward',
    'time_closest_to_ball',
    'time_farthest_from_ball',
#     'goals_against_while_last_defender'
]

In [9]:
boay.drop(columns = to_drop, inplace = True)
cho.drop(columns = to_drop, inplace = True)

In [11]:
for col in boay.columns[:-1]:
    boay[col] = boay[col].astype(float)

In [19]:
for col in cho.columns[:-1]:
    cho[col] = cho[col].astype(float)

In [14]:
cho.drop(columns = 'goals_against_while_last_defender', inplace = True)

In [27]:
cho_preds = pd.DataFrame(et_model.predict_proba(cho.drop(columns = 'player_name')), columns = ['Monkeymoon','Oski','Vati'])

In [28]:
cho_preds

,Monkeymoon,Oski,Vati
0,0.534330,0.088019,0.377651
1,0.534330,0.088019,0.377651
2,0.517357,0.132630,0.350013
3,0.411180,0.091033,0.497787
4,0.466278,0.206499,0.327223
5,0.440480,0.146537,0.412983
6,0.455669,0.317467,0.226864
7,0.583656,0.156698,0.259646
8,0.593985,0.202664,0.203350
9,0.508891,0.142663,0.348445


In [22]:
cho_preds['Playstyle'] = 'Monkeymoon'

In [23]:
cho_preds

,Monkeymoon,Oski,Vati,Playstyle
0,0.534330,0.088019,0.377651,Monkeymoon
1,0.534330,0.088019,0.377651,Monkeymoon
2,0.517357,0.132630,0.350013,Monkeymoon
3,0.411180,0.091033,0.497787,Monkeymoon
4,0.466278,0.206499,0.327223,Monkeymoon
5,0.440480,0.146537,0.412983,Monkeymoon
6,0.455669,0.317467,0.226864,Monkeymoon
7,0.583656,0.156698,0.259646,Monkeymoon
8,0.593985,0.202664,0.203350,Monkeymoon
9,0.508891,0.142663,0.348445,Monkeymoon


Monkeymoon    0.616272
Oski          0.317467
Vati          0.530441
dtype: float64

In [20]:
nn_model.predict(cho.drop(columns = 'player_name'))

1/1 [==============================] - 0s 145ms/step


array([[0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 0.99999994, 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ]], dtype=float32)